In [1]:
#export
import pandas as pd
import numpy as np
import json

In [37]:
DATA_LOC = 'https://github.com/offentlig-ai/nace-notebooks/raw/master/data/'
DATA_STORE = '/data/'

## Standard for næringsgruppering (SN)
kilde: https://www.ssb.no/klass/klassifikasjoner/6

In [3]:
nace_2007 = pd.read_csv(DATA_LOC + '30.csv', sep=';', encoding='iso-8859-1') 
nace_2007.head(1)

,code,parentCode,level,name,shortName,notes,validFrom,validTo
0,01,A,2,"Jordbruk og tjenester tilknyttet jordbruk, jak...","Jordbruk, tilhør. tjenester, jakt",Inkluderer: Denne næringen omfatter to basisak...,NaN,NaN


## NACE (næringsgruppe) koder pr foretak

In [4]:
nace = pd.read_csv(DATA_LOC + 'EnhetNacekode.tab', sep='\t', encoding='utf-8', dtype={'nacekode': object}) 
nace_pivot = pd.pivot_table(nace.sort_values(by=['orgnr', 'rekkefolge']),
                            index='orgnr', columns='rekkefolge',
                            values='nacekode', aggfunc='first').reset_index()
nace_pivot.columns=['orgnr', 'nace_1', 'nace_2', 'nace_3','nace_4']
nace_pivot.head(1)

,orgnr,nace_1,nace_2,nace_3,nace_4
0,810033622,68.209,NaN,NaN,NaN


In [5]:
len(nace_pivot)

1070788

In [6]:
enhet = pd.read_csv(DATA_LOC + 'EnhetNavnOrgform.tab', sep='\t', encoding='iso-8859-1') 
enhet.head()

,orgnr,orgform,navn
0,810033622,AS,WILTHIL & CO AS
1,810034882,AS,SANDNES ELEKTRISKE FORRETNING AS
2,810059672,AS,AASEN & FARSTAD AS
3,810093382,AS,BRIS EIENDOM AS
4,810094532,AS,AGDERPOSTEN MEDIER AS


In [7]:
len(enhet)

1124902

## Beskrivelser for NACE kodene

In [8]:
import requests
from io import StringIO
r = requests.get('http://data.ssb.no/api/klass/v1/classifications/6/codes?from=1990-01-01&to=2019-01-01')
nace_beskrivelse = pd.read_csv(StringIO(r.text))
nace_beskrivelse = nace_beskrivelse[nace_beskrivelse['level']==6]
nace_beskrivelse = nace_beskrivelse.sort_values(['code','validFromInRequestedRange']
                                                ,ascending=False).groupby('code').head(1)[['code','name']]

In [9]:
#nace_beskrivelse = pd.read_csv(DATA_LOC + 'nace_beskrivelse.tab', sep='\t', encoding='utf-8', dtype={'nacekode': object}) 
#nace_beskrivelse.drop(['beskrivelse2'], axis=1, inplace=True)
#nace_beskrivelse.columns = ['nace', 'tekst']

In [10]:
#nace_beskrivelse.head()

In [11]:
#nace_beskrivelse.to_json(DATA_LOC + 'nace_beskrivelse.json', orient="records")

## Formålet med virksomheten beskrevet i tekst

In [12]:
formål = pd.read_csv(DATA_LOC + 'EnhetOrgformVirksomhetArtBransje.tab', sep='\t', encoding='iso-8859-1') 
formål.columns = ['orgnr', 'orgform', 'linje_nr', 'linje_tekst']

In [13]:
len(formål)

1901937

In [14]:
formål.head(5)

,orgnr,orgform,linje_nr,linje_tekst
0,810033622,AS,1,"Handel, engros og detalj og agenturer, herunder"
1,810033622,AS,2,import av varer.
2,810034882,AS,10,"Handel og innstallasjonsvirksomhet, eller anne..."
3,810034882,AS,20,"med dette, samt delta I annen virksomhet."
4,810059672,AS,10,Utleie av forretningseiendommer.


In [15]:
# eksempel: før sammenslåing
list(formål[formål['orgnr'] == 810034882]['linje_tekst'])

['Handel og innstallasjonsvirksomhet, eller annen virksomhet forbundet',
 'med dette, samt delta I annen virksomhet.']

In [16]:
# eksempel: etter sammenslåing
formål_t = formål[['orgnr','linje_tekst']].groupby('orgnr')['linje_tekst'].apply(lambda x: ' '.join(x)).reset_index()

In [17]:
list(formål_t[formål_t['orgnr'] == 810034882]['linje_tekst'])

['Handel og innstallasjonsvirksomhet, eller annen virksomhet forbundet med dette, samt delta I annen virksomhet.']

In [18]:
len(formål_t)

1102419

In [19]:
formål_t.head()

,orgnr,linje_tekst
0,810033622,"Handel, engros og detalj og agenturer, herunde..."
1,810034882,"Handel og innstallasjonsvirksomhet, eller anne..."
2,810059672,Utleie av forretningseiendommer.
3,810093382,"Eie og drift av fast eiendom, investering i ak..."
4,810094532,Utgi avisen Agderposten og drive annen virksom...


In [20]:
df = pd.merge(formål_t, nace_pivot, on='orgnr', how='inner') 

In [21]:
# TODO: drop konkurser, ukjent osv

In [22]:
len(df)

1049701

In [23]:
df['lengde_linje_tekst'] = df['linje_tekst'].apply(lambda x: len(x))
df = df[df['lengde_linje_tekst']>4]

In [24]:
len(df)

1048162

## Datasett for maskinlæring

In [25]:
"""
import spacy
import re

nlp = spacy.load('nb_ud_ner')

def tokenize(x):
    tokens = nlp.tokenizer(x)
    return [t.text.lower() for t in tokens if not t.is_punct]
"""

"\nimport spacy\nimport re\n\nnlp = spacy.load('nb_ud_ner')\n\ndef tokenize(x):\n    tokens = nlp.tokenizer(x)\n    return [t.text.lower() for t in tokens if not t.is_punct]\n"

In [26]:
# simplified tokenization
def tokenize(x):
    tokens = x.split(' ')
    return [t.lower() for t in tokens]

In [27]:
# test
test = 'Handel og innstallasjonsvirksomhet, eller annen virksomhet forbundet med dette, samt delta I annen virksomhet.'
tokenize(test)

['handel',
 'og',
 'innstallasjonsvirksomhet,',
 'eller',
 'annen',
 'virksomhet',
 'forbundet',
 'med',
 'dette,',
 'samt',
 'delta',
 'i',
 'annen',
 'virksomhet.']

In [28]:
#export
def get_dataset(row):
    s = row['linje_tekst']
    tokens = tokenize(s)
    tokens.append(f"__label__{row['nace_1']}")
    linje = ' '.join(tokens)
    return linje

In [29]:
dataset = df.apply(get_dataset, axis=1)

In [30]:
dataset.head(5)

0    handel, engros og detalj og agenturer, herunde...
1    handel og innstallasjonsvirksomhet, eller anne...
2     utleie av forretningseiendommer. __label__68.209
3    eie og drift av fast eiendom, investering i ak...
4    utgi avisen agderposten og drive annen virksom...
dtype: object

In [31]:
len(dataset)

1048162

In [32]:
dataset.iloc[0]

'handel, engros og detalj og agenturer, herunder import av varer. __label__68.209'

In [33]:
# oppdeling av datasettet i 90% trening, 10% validering
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.1)
#train, test = np.split(dataset.sample(frac=1), [int(.9*len(dataset))])

In [38]:
train.to_csv(DATA_STORE + 'næringskoder_train.txt',
                                       header = None,
                                      index = None,
                                      sep = ' ', encoding='utf-8')

/Users/B149899/anaconda3/envs/dv/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


FileNotFoundError: [Errno 2] No such file or directory: '/data/næringskoder_train.txt'

In [ ]:
test.to_csv(DATA_STORE + 'næringskoder_test.txt',
                                       header = None,
                                      index = None,
                                      sep = ' ', encoding='utf-8')

## Trene modell 

In [ ]:
#export
import fastText
from fastText import train_supervised, tokenize, load_model, unicode_literals

In [ ]:
# list metoder
# dir(fastText)

In [ ]:
#export
model = train_supervised(
    input = DATA_STORE + 'næringskoder_train.txt',
    wordNgrams = 3,
    label = '__label__',
    verbose = 2,
    minCount = 1,
    neg = 10
)

In [ ]:
# list metoder
# dir(model)

In [ ]:
model.save_model(DATA_STORE + 'nace_model.bin')

In [ ]:
model.quantize(DATA_STORE + 'nace_model_quantized.bin')

In [ ]:
#export
def print_results(N,p,r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1,p))
    print("R@{}\t{:.3f}".format(1,r))

In [ ]:
print_results(*model.test(DATA_LOC + 'næringskoder_validate.txt'))

In [ ]:
#export
def predict_nace(model, tekst, k):
    res = {}
    res['nace'] = []
    pred = model.predict(tekst, k=k)
    for index, item in enumerate(pred[0]):
        res['nace'].append({'nace': item.replace('__label__', ''), 'prob': pred[1][index] })
    return res

In [ ]:
tekst = test.iloc[400][:-16]
print(tekst)

In [ ]:
predict_nace(model, tekst, 5)

In [ ]:
test.values

In [ ]:
test.values[:10]

In [ ]:
pred_list=[]
for index, row in enumerate(test.values):
    nace = row[-6:]
    line = row[:-16]
    length = len(line)
    if (length > 1):
        pred = predict_nace(model, line,3)
        pred_1 = pred['nace'][0]['nace'][-7:-1]
        pred_2 = pred['nace'][1]['nace'][-7:-1]
        pred_3 = pred['nace'][2]['nace'][-7:-1]

        korrekt = 'Nei'
        if pred_1 == nace:
            korrekt = 'Ja'

        if nace in [pred_2,pred_3]:
            korrekt = 'Delvis'

        pred_list.append([index, nace, korrekt, pred_1, pred_2, pred_3,line])
    else:
        print(row)
        
pred = pd.DataFrame(pred_list)
pred.columns=['index','nace','korrekt','pred_1','pred_2','pred_3','tekst']

In [ ]:
# 10% split
len(pred)

In [ ]:
pred.head()

In [ ]:
resultat = pred[['index','nace','korrekt']].groupby(['nace','korrekt']).count().reset_index().rename(columns={'index':'count'})

In [ ]:
resultat.head()

In [ ]:
pv = pd.pivot_table(resultat, index='nace', columns='korrekt').reset_index()

In [ ]:
pv = pd.pivot_table(resultat, index='nace', columns='korrekt').reset_index()
pv.fillna(0, inplace = True)
pv.columns = ['nace', 'delvis_korrekt','korrekt','feil']
pv['antall'] = pv['feil'] + pv['korrekt'] + pv['delvis_korrekt']
pv['andel_korrekt'] = 100 * pv['korrekt'] / pv['antall']
pv['andel_feil'] = 100 * pv['feil'] / pv['antall']
pv['andel_delvis_korrekt'] = 100 * (pv['korrekt'] + pv['delvis_korrekt']) / pv['antall']
pv.sort_values(by='andel_feil', ascending = False, inplace=True) 

In [ ]:
count = resultat[['nace','count']].groupby('nace').sum().reset_index()
pv = pd.merge(pv, count, how='left', on='nace')

## Andel korrekte fastsettelser

In [ ]:
pv_t = pd.merge(pv,nace_beskrivelse, how='left', on='nace')
pv_t.rename(columns={'name':'nace_beskrivelse'}, inplace=True)

### Høy treffsikkerhet

In [ ]:
pv_t.sort_values(by='andel_feil', ascending = True, inplace=True)
pv_t.head(20)

### Lav treffsikkerhet

In [ ]:
pv_t.tail(20)

In [ ]:
plot = pv_t.vgplot.scatter(x='antall', y='andel_korrekt', c='andel_delvis_korrekt')
plot.display()

In [ ]:
#plot.spec

In [ ]:
pv_t.vgplot.scatter(x='antall', y='andel_delvis_korrekt', c='andel_korrekt')

In [ ]:
pv_t['sum_antall'] = pv_t['antall'].cumsum()

In [ ]:
pv_t.vgplot.scatter(x='andel_feil', y='sum_antall', c='andel_korrekt')

In [ ]:
pv_t['nace_hoved'] = pv_t['nace'].apply(lambda x: x[0:2])

In [ ]:
pv_t.head(1)

In [ ]:
pvh = pv_t[['nace_hoved','antall','korrekt','delvis_korrekt','feil']].groupby(['nace_hoved']).sum().reset_index()
pvh = pvh[pvh['antall']>0]
pvh['andel_feil'] = 100*pvh['feil']/pvh['antall']

In [ ]:
pvh = pd.merge(pvh, nace_2007[['code','name']], how="left", left_on='nace_hoved', right_on='code')
pvh.rename(columns={'name':'nace_hoved_beskrivelse'}, inplace=True)

In [ ]:
pvh.sort_values(by='andel_feil', ascending = True, inplace=True) 

In [ ]:
pvh.head(20)

In [ ]:
pvh.tail(20)

In [ ]:
pvh['sum_antall'] = pvh['antall'].cumsum()

In [ ]:
pvh.vgplot.scatter(x='andel_feil', y='sum_antall')

In [ ]:
pred['nace_hoved'] = pred['nace'].apply(lambda x: x[0:2])
data = pd.merge(pred,nace_beskrivelse.rename(columns={'tekst':'nace_beskrivelse'}), how='left', on='nace')
data.head(10)

In [ ]:
data = pd.merge(data,nace_beskrivelse.rename(columns={'tekst':'nace_beskrivelse'}), how='left', left_on='pred_1', right_on='nace', suffixes=['','_pred_1'])
data = pd.merge(data,nace_beskrivelse.rename(columns={'tekst':'nace_beskrivelse'}), how='left', left_on='pred_2', right_on='nace', suffixes=['','_pred_2'])
data = pd.merge(data,nace_beskrivelse.rename(columns={'tekst':'nace_beskrivelse'}), how='left', left_on='pred_3', right_on='nace', suffixes=['','_pred_3'])

In [ ]:
from IPython.display import display
display(data)

In [ ]:
print(f"Antall: {len(data[data['nace']=='46.150'])}")
data[data['nace_hoved']=='13']

In [ ]:
data[data['nace_hoved']=='61']

In [ ]:
data[data['nace_hoved']=='06']

In [ ]:
print(f"Antall: {len(data[data['nace']=='94.991'])}")
data[data['nace']=='94.991'].head(20)

In [ ]:
writer = pd.ExcelWriter(DATA_STORE + 'prediksjon.xlsx')
data.to_excel(writer,'prediksjon')
writer.save()